In [1]:
!pip install -q transformers
!pip install pydot
!pip install tensorflow==2.11.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 6.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 71.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 102.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 77.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 100.5 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tenso

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from collections import Counter
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pprint import pprint 
from transformers import BertTokenizer, TFBertModel

In [3]:
model_checkpoint = 'bert-base-cased'
bert_tokenizer = BertTokenizer.from_pretrained(model_checkpoint)
bert_model = TFBertModel.from_pretrained(model_checkpoint)

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [4]:
max_length = 200

In [5]:
train = pd.read_csv('train_data_imbalanced_stratified_no_dupe_17k_5k.csv')
val = pd.read_csv("validation_data_imbalanced_stratified_no_dupe_17k_5k.csv")
test = pd.read_csv("test_data_imbalanced_stratified_no_dupe.csv")

In [6]:
train_text = train['text'].tolist()
val_text = val['text'].tolist()
test_text = test['text'].tolist()

In [7]:
train_encodings = bert_tokenizer(train_text, truncation=True, padding='max_length', max_length=max_length, return_tensors='tf')
valid_encodings = bert_tokenizer(val_text, truncation=True, padding='max_length', max_length=max_length, return_tensors='tf')
test_encodings = bert_tokenizer(test_text, truncation=True, padding='max_length', max_length=max_length, return_tensors='tf')

In [8]:
train_labels = pd.Categorical(train['emotions']).codes
npval_labels = pd.Categorical(val['emotions']).codes
nptest_labels = pd.Categorical(test['emotions']).codes

In [9]:
def create_bert_multiclass_model(checkpoint = model_checkpoint,
                                 num_classes = 7,
                                 hidden_size = 201, 
                                 dropout=0.3,
                                 learning_rate=0.00005):
    bert_model = TFBertModel.from_pretrained(checkpoint)                                              
    max_length = 200
    bert_model.trainable = True
    input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name='input_ids_layer')
    token_type_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name='token_type_ids_layer')
    attention_mask = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name='attention_mask_layer')
    bert_inputs = {'input_ids': input_ids,
                   'token_type_ids': token_type_ids,
                   'attention_mask': attention_mask}    
    bert_out = bert_model(bert_inputs)
    cls_token = bert_out[0][:, 0, :]
    hidden = tf.keras.layers.Dense(hidden_size, activation='relu', name='hidden_layer')(cls_token)
    hidden = tf.keras.layers.Dropout(dropout)(hidden) 
    classification = tf.keras.layers.Dense(num_classes, activation='softmax',name='classification_layer')(hidden)
    classification_model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=[classification])
    classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                                 loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), 
                                 metrics='accuracy') 
    return classification_model

### Part 1: Experiment with different hidden sizes

In [10]:
mod_1 = create_bert_multiclass_model(checkpoint=model_checkpoint, num_classes=7, hidden_size = 50)
mod_2 = create_bert_multiclass_model(checkpoint=model_checkpoint, num_classes=7, hidden_size = 100)
mod_3 = create_bert_multiclass_model(checkpoint=model_checkpoint, num_classes=7, hidden_size = 150)
mod_4 = create_bert_multiclass_model(checkpoint=model_checkpoint, num_classes=7, hidden_size = 300)
mod_5 = create_bert_multiclass_model(checkpoint=model_checkpoint, num_classes=7, hidden_size = 250)
mod_6 = create_bert_multiclass_model(checkpoint=model_checkpoint, num_classes=7)


Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.
Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initiali

In [ ]:
mod_1_history = mod_1.fit([train_encodings.input_ids, train_encodings.token_type_ids, train_encodings.attention_mask], 
                                                  train_labels,   
                                                  validation_data=([valid_encodings.input_ids, valid_encodings.token_type_ids, valid_encodings.attention_mask], 
                                                  npval_labels),    
                                                  batch_size=8, 
                                                  epochs=2)  

Epoch 1/2


2136/2136 [==============================] - 839s 368ms/step - loss: 0.3468 - accuracy: 0.8985 - val_loss: 0.1309 - val_accuracy: 0.9575
Epoch 2/2
2136/2136 [==============================] - 780s 365ms/step - loss: 0.1195 - accuracy: 0.9697 - val_loss: 0.1099 - val_accuracy: 0.9760


In [ ]:
mod_2_history = mod_2.fit([train_encodings.input_ids, train_encodings.token_type_ids, train_encodings.attention_mask], 
                                                  train_labels,   
                                                  validation_data=([valid_encodings.input_ids, valid_encodings.token_type_ids, valid_encodings.attention_mask], 
                                                  npval_labels),    
                                                  batch_size=8, 
                                                  epochs=2) 

Epoch 1/2


2136/2136 [==============================] - 840s 368ms/step - loss: 0.3042 - accuracy: 0.9093 - val_loss: 0.0619 - val_accuracy: 0.9811
Epoch 2/2
2136/2136 [==============================] - 780s 365ms/step - loss: 0.1096 - accuracy: 0.9691 - val_loss: 0.0782 - val_accuracy: 0.9754


In [ ]:
mod_3_history = mod_3.fit([train_encodings.input_ids, train_encodings.token_type_ids, train_encodings.attention_mask], 
                                                  train_labels,   
                                                  validation_data=([valid_encodings.input_ids, valid_encodings.token_type_ids, valid_encodings.attention_mask], 
                                                  npval_labels),    
                                                  batch_size=8, 
                                                  epochs=2) 

Epoch 1/2


2136/2136 [==============================] - 827s 365ms/step - loss: 0.2927 - accuracy: 0.9122 - val_loss: 0.0736 - val_accuracy: 0.9793
Epoch 2/2
2136/2136 [==============================] - ETA: 0s - loss: 0.0951 - accuracy: 0.9746

In [11]:
mod_4_history = mod_4.fit([train_encodings.input_ids, train_encodings.token_type_ids, train_encodings.attention_mask], 
                                                  train_labels,   
                                                  validation_data=([valid_encodings.input_ids, valid_encodings.token_type_ids, valid_encodings.attention_mask], 
                                                  npval_labels),    
                                                  batch_size=8, 
                                                  epochs=2) 

Epoch 1/2


2136/2136 [==============================] - 890s 391ms/step - loss: 0.2765 - accuracy: 0.9187 - val_loss: 0.0983 - val_accuracy: 0.9703
Epoch 2/2
2136/2136 [==============================] - 822s 385ms/step - loss: 0.0967 - accuracy: 0.9721 - val_loss: 0.0946 - val_accuracy: 0.9701


In [12]:
mod_5_history = mod_5.fit([train_encodings.input_ids, train_encodings.token_type_ids, train_encodings.attention_mask], 
                                                  train_labels,   
                                                  validation_data=([valid_encodings.input_ids, valid_encodings.token_type_ids, valid_encodings.attention_mask], 
                                                  npval_labels),    
                                                  batch_size=8, 
                                                  epochs=2) 

Epoch 1/2


2136/2136 [==============================] - 882s 387ms/step - loss: 0.2974 - accuracy: 0.9104 - val_loss: 0.0772 - val_accuracy: 0.9787
Epoch 2/2
2136/2136 [==============================] - 821s 385ms/step - loss: 0.0941 - accuracy: 0.9744 - val_loss: 0.0850 - val_accuracy: 0.9754


In [13]:
mod_6_history = mod_6.fit([train_encodings.input_ids, train_encodings.token_type_ids, train_encodings.attention_mask], 
                                                  train_labels,   
                                                  validation_data=([valid_encodings.input_ids, valid_encodings.token_type_ids, valid_encodings.attention_mask], 
                                                  npval_labels),    
                                                  batch_size=8, 
                                                  epochs=2) 

Epoch 1/2


2136/2136 [==============================] - 880s 387ms/step - loss: 0.3031 - accuracy: 0.9103 - val_loss: 0.1266 - val_accuracy: 0.9666
Epoch 2/2
2136/2136 [==============================] - 822s 385ms/step - loss: 0.0952 - accuracy: 0.9733 - val_loss: 0.0619 - val_accuracy: 0.9817


There is not much different in loss between the models tested, however model 6 has the highest validation accuracy and lowest loss, so we use this going forward.